BCI Sample Size Determination (SSD)

In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm
from scipy import special

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# sample size
n_subjects = [4, 8, 12, 16, 24] # number of subjects
n_trials = [20, 36, 64] # number of trials

M = 500 # iterations

# MCMC
n_samples = 5000
n_chains = 3
n_tune = 500

In [ ]:
for Ns in n_subjects:
    ALCs = []
    for T in n_trials:
        ALC = 0
        for m in range(M):
            print("M: "+str(m)+"\t Ns: "+str(Ns)+"\t T: "+str(T))
            
            # draw parameters (theta hat) from sampling prior
            print("Step 1: draw parameters from sampling prior")
            mu_phi = np.random.uniform(0.55, 0.95)
            mu_alpha = special.logit(mu_phi)
            sigma_alpha = np.random.uniform(0.2, 1.2)
            
            alpha = np.random.normal(mu_alpha, sigma_alpha, Ns)
            phi = special.expit(alpha)
            
            # draw dataset D^(n) from sampling distribution
            print("Step 2: draw data from sampling distribution")
            y = np.random.binomial(T, phi) # vector of size Ns
            
            # compute delta(D^(n)) using Baye's rule (via MCMC)
            print("Step 3: compute delta using Baye's rule (via MCMC)")
            with pm.Model() as model:
                # priors for group level parameters - a single value for mean and std
                group_level_mean_logit = pm.Normal('μ_α', mu=0, sd=2**0.5)
                group_level_std_logit = pm.Uniform('σ_α', lower=0., upper=10.)
                
                group_level_mean_prob = pm.Deterministic('μ_φ', pm.math.invlogit(group_level_mean_logit))
                
                # subject level parameters - vector of size Ns
                subject_level_accuracy_logit = pm.Normal('α', mu=group_level_mean_logit, sd=group_level_std_logit, shape=Ns)
                subject_level_accuracy_prob = pm.Deterministic('φ', pm.math.invlogit(subject_level_accuracy_logit))
                
                # likelihood (sampling distributions) of observations
                y_obs = []
                for s in range(Ns):
                    y_obs_i = pm.Binomial('y_obs_'+str(s), n=T, p=subject_level_accuracy_prob[s], observed=y[s])
                    y_obs.append(y_obs_i)
                
                # draw posterior samples
                trace = pm.sample(n_samples, chains=n_chains, tune=n_tune, discard_tuned_samples=True)
            
            # posterior analysis
            trace_np = pd.DataFrame(trace['α']).to_numpy() # 15000 x Ns array of subject-level accuracies
            
            # take mean across all Ns subjects --> vector of 15000
            group_level_mean_logit_hat = np.mean(trace_np, axis=1)
            
            # compute 95% CI
            delta = np.percentile(group_level_mean_logit_hat, 97.5) - np.percentile(group_level_mean_logit_hat, 2.5)
            print("delta:", delta)

            # approximate ALC
            ALC += delta
        ALC /= M
        print("ALC:", ALC)
        ALCs.append(ALC)
    plt.plot(n_trials, ALCs, marker='x', label='Ns = '+str(Ns))

plt.xlabel('Number of trials T')
plt.ylabel('Average 95% CI width ALC(n)')
plt.legend()
plt.show()

In [ ]:
pm.traceplot(trace);